# Selenium

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

# Setup Chrome options
options = Options()
# options.add_argument("--start-maximized")  # Optional: Maximize window
# options.add_argument("--headless")  # Optional: Run headless if GUI is not needed

# Install & initialize WebDriver automatically|
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Set implicit wait
driver.implicitly_wait(10)

print("WebDriver initialized successfully!")


WebDriver initialized successfully!


In [ ]:
# url = 'https://property.franklincountyauditor.com/_web/search/commonsearch.aspx?mode=condo_subdiv'
# url = 'https://www.rebny.com/members/'
# response = requests.get('https://filmfreeway.com/CERVINOCINEMOUNTAIN')
# url = "https://filmfreeway.com/CERVINOCINEMOUNTAIN"
# url = "https://puppychoices.co.uk/breeder/amazing-cocker-spaniels/"
# url = 'https://www.officedepot.com/b/office-chairs/N-593067'
# url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Florida"
# url = "https://barreaubruxelles.be/annuaire/p1#results"

# url ="https://www.myfloridalicense.com/wl11.asp?mode=2&search=LicTyp&SID=&brd=&typ=N"
url = "https://frontier.com/"
driver.get(url)


# Requests

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Define the headers as seen in the request
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9,bn;q=0.8",
    "Origin": "https://map.gulfood.com",
    "Referer": "https://map.gulfood.com/",
    "sec-ch-ua": '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
    "x-map-id": "obl0gRq3GXBm1"
}

# # url = "https://www.g2.com/products/salesforce-salesforce-sales-cloud/reviews?page=1"
# url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Florida"

# url = "https://www.firmenabc.at/traupmann-cnc-elektronik-gmbh_DYuk"
url = "https://mtu-limbach.de/S-N-Leukostrip-steril-50-x-3-Streifen/66002883"
response = requests.get(url, headers=headers)





In [ ]:
soup = BeautifulSoup(response.text, 'lxml')

# New Work -> BLS Scraping

In [ ]:
# Please follow the following steps:

# Step 1: Go to the BLS data website (https://www.bls.gov/lau/data.htm)

# Step 2: click on "multi-screen"

# Step 3: select all states or several states at a time 

# Step 4: select "Counties and equivalents"

# Step 5: select all areas shown in the box

# Step 6: select "unemployment rate" 

# Step 7: select "Not Seasonally Adjusted" --> then click on "retrieve data".

In [32]:
# Step 1: Multi-Screen
driver.get('https://data.bls.gov/multi-screen?survey=la')

In [43]:
# all_states_elm = driver.find_elements(By.XPATH, '//select[@id="selectbox-single"]/option')
# for elm in all_states_elm:
#     state_name = elm.text
#     print(f"Processing state: {state_name}")
#     # elm.click()
#     # time.sleep(2)

## Get County List

In [48]:
records = []

In [ ]:
for i in range(0, 52):
    driver.get('https://data.bls.gov/multi-screen?survey=la')
    time.sleep(1)

    all_states_elm = driver.find_elements(By.XPATH, '//select[@id="selectbox-single"]/option')

    ## Select State/Region
    state_selected = all_states_elm[i].text
    print(f"{state_selected}")
    all_states_elm[i].click()

    ## Click - Next form
    driver.find_element(By.XPATH, '//button[@id="sa-next-button"]').click()
    time.sleep(1)

    ## Select [OPTION]- Counties and equivalents
    # driver.find_elements(By.XPATH, '//select[@id="selectbox-single"]/option')[4].click()
    # time.sleep(1)
    # Find all the options inside the select
    options = driver.find_elements(By.XPATH, '//select[@id="selectbox-single"]/option')

    # Loop through the options and select the one that matches
    found = False
    for option in options:
        if "F Counties and equivalents" in option.text:
            option.click()
            found = True
            time.sleep(1)
            break
    
    if found == False:
        print(f'\n[CAUTION]: Counties and equivalents option not found.\n')
        continue
    ## Click - Next form
    driver.find_element(By.XPATH, '//button[@id="sa-next-button"]').click()
    time.sleep(1)

    county_elements = driver.find_elements(By.XPATH, '//select[@class="form-select ng-untouched ng-pristine ng-invalid"]/option')
    for el in county_elements:
        info = [state_selected, el.text]
        if info not in records:
            records.append(info)

    print(f'Total Counties Found on {state_selected}: {len(county_elements)} | Records: {len(records)}')

    # break

## Saving the DataFrame
df = pd.DataFrame(records, columns=['State', 'County'])

# Step 1: Split into Code + Rest
df[['County_Code', 'Rest']] = df['County'].str.split(' ', n=1, expand=True)

# Step 2: Split Rest into Name + State (renaming extracted State to avoid overwrite)
df[['County_Name', 'County_State']] = df['Rest'].str.rsplit(',', n=1, expand=True)

# Step 3: Clean whitespace
df['County_Name'] = df['County_Name'].str.strip()
df['County_State'] = df['County_State'].str.strip()

# Drop the helper column
df = df.drop(columns=['Rest'])
df.to_excel('bls_counties_list.xlsx', index=False)

01 Alabama
Total Counties Found on 01 Alabama: 67 | Records: 67
02 Alaska
Total Counties Found on 02 Alaska: 34 | Records: 101
04 Arizona
Total Counties Found on 04 Arizona: 15 | Records: 116
05 Arkansas
Total Counties Found on 05 Arkansas: 75 | Records: 191
06 California
Total Counties Found on 06 California: 58 | Records: 249
08 Colorado
Total Counties Found on 08 Colorado: 64 | Records: 313
09 Connecticut
Total Counties Found on 09 Connecticut: 9 | Records: 322
10 Delaware
Total Counties Found on 10 Delaware: 3 | Records: 325
11 District of Columbia
Total Counties Found on 11 District of Columbia: 1 | Records: 326
12 Florida
Total Counties Found on 12 Florida: 67 | Records: 393
13 Georgia
Total Counties Found on 13 Georgia: 159 | Records: 552
15 Hawaii
Total Counties Found on 15 Hawaii: 4 | Records: 556
16 Idaho
Total Counties Found on 16 Idaho: 44 | Records: 600
17 Illinois
Total Counties Found on 17 Illinois: 102 | Records: 702
18 Indiana
Total Counties Found on 18 Indiana: 92 | R

In [74]:
def convert_county_code(county_code: str) -> str:
    """
    Convert a county code like 'CN4202500000000' into 
    a BLS series ID like 'LAUCN420250000000003'.

    Args:
        county_code (str): County code starting with 'CN'

    Returns:
        str: Converted BLS series ID
    """
    return f"LAU{county_code}03"

In [75]:
df['dm_code'] = df['County_Code'].apply(convert_county_code)

### Functions In Use

In [ ]:
import requests

def fetch_bls_series(series_id: str):
    """
    Fetch BLS data for a given series_id from SurveyOutputServlet.
    
    Args:
        series_id (str): The BLS series ID (e.g., 'LAUCN010010000000003').
    
    Returns:
        dict: JSON response from the API
    """
    url = "https://data.bls.gov/pdq/SurveyOutputServlet"

    payload = {
        "request_action": "get_data",
        "reformat": "true",
        "from_results_page": "true",
        "from_year": "1990",
        "to_year": "2025",
        "Go.x": "9",
        "Go.y": "3",
        "initial_request": "false",
        "data_tool": "dsrv",
        "series_id": series_id,
        "output_type": "simple",
        "original_output_type": "simple",
        "original_annualAveragesRequested": "false"
    }

    response = requests.post(url, data=payload)
    response.raise_for_status()
    return response

def parse_catalog_info(rows):
    """
    Parse catalog information from <tr> elements into structured variables.
    
    Args:
        rows (list): List of BeautifulSoup <tr> elements.
    
    Returns:
        dict: Dictionary with structured variables.
    """
    # Initialize variables as None (so we know if something is missing)
    series_id = None
    area = None
    area_type = None
    state_region_division = None
    measure = None
    seasonal = None

    for row in rows:
        th = row.find("th")
        td = row.find("td")

        if th and td:
            key = th.get_text(strip=True).replace(":", "")
            value = td.get_text(strip=True)

            if key == "Series Id":
                series_id = value
            elif key == "Area":
                area = value
            elif key == "Area Type":
                area_type = value
            elif key == "State/Region/Division":
                state_region_division = value
            elif key == "Measure":
                measure = value
        elif td and not th:
            # Handles rows like "Not Seasonally Adjusted"
            seasonal = td.get_text(strip=True)

    return {
        "Series_Id": series_id,
        "Seasonal": seasonal,
        "Area": area,
        "Area_Type": area_type,
        "State_Region_Division": state_region_division,
        "Measure": measure
    }

def get_table_df(response):
    soup = BeautifulSoup(response.text, 'lxml')
    basic_info = parse_catalog_info(soup.select('#catalog0 tr'))

    # Extract the HTML table as a string
    table_html = str(soup.select_one('#table0'))

    # Parse with pandas
    sf = pd.read_html(table_html)[0]

    # Remove the last row
    sf = sf.iloc[:-1]


    ## Final Work
    sf['series_id'] = basic_info['Series_Id']
    sf['Area'] = basic_info['Area']
    sf['Area_Type'] = basic_info['Area_Type']
    sf['State_Region_Division'] = basic_info['State_Region_Division']
    sf['Measure'] = basic_info['Measure']

    sf = sf[['series_id', 'Area', 'Area_Type', 'State_Region_Division', 'Measure', 'Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

    return sf

In [78]:
df.head()

,State,County,County_Code,County_Name,County_State,dm_code
0,01 Alabama,"CN0100100000000 Autauga County, AL",CN0100100000000,Autauga County,AL,LAUCN010010000000003
1,01 Alabama,"CN0100300000000 Baldwin County, AL",CN0100300000000,Baldwin County,AL,LAUCN010030000000003
2,01 Alabama,"CN0100500000000 Barbour County, AL",CN0100500000000,Barbour County,AL,LAUCN010050000000003
3,01 Alabama,"CN0100700000000 Bibb County, AL",CN0100700000000,Bibb County,AL,LAUCN010070000000003
4,01 Alabama,"CN0100900000000 Blount County, AL",CN0100900000000,Blount County,AL,LAUCN010090000000003


In [104]:
dfs = []

for i in range(20):
    code = df.at[i, 'dm_code']
    response = fetch_bls_series(code)
    sf = get_table_df(response)

    dfs.append(sf)

    print(f'{i}: {code}')

/var/folders/j7/kkj6r3cn19q44jwd34kcpjnh0000gn/T/ipykernel_1515/2960171742.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sf = pd.read_html(table_html)[0]


0: LAUCN010010000000003


/var/folders/j7/kkj6r3cn19q44jwd34kcpjnh0000gn/T/ipykernel_1515/2960171742.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sf = pd.read_html(table_html)[0]


1: LAUCN010030000000003


/var/folders/j7/kkj6r3cn19q44jwd34kcpjnh0000gn/T/ipykernel_1515/2960171742.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sf = pd.read_html(table_html)[0]


...

/var/folders/j7/kkj6r3cn19q44jwd34kcpjnh0000gn/T/ipykernel_1515/2960171742.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sf = pd.read_html(table_html)[0]


17: LAUCN010350000000003


/var/folders/j7/kkj6r3cn19q44jwd34kcpjnh0000gn/T/ipykernel_1515/2960171742.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sf = pd.read_html(table_html)[0]


18: LAUCN010370000000003
19: LAUCN010390000000003


/var/folders/j7/kkj6r3cn19q44jwd34kcpjnh0000gn/T/ipykernel_1515/2960171742.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sf = pd.read_html(table_html)[0]


In [105]:

wf = pd.concat(dfs)

In [107]:
wf.to_excel('bls_counties_data_sample.xlsx', index=False)

,series_id,Area,Area_Type,State_Region_Division,Measure,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1990,6.5,6.4,5.6,6.6,6.1,7.2,6.0,6.8,7.3,7.3,6.3,6.3
1,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1991,6.5,7.3,6.9,6.5,6.4,7.6,6.6,7.2,6.9,6.3,6.1,6.1
2,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1992,6.8,6.8,6.3,5.6,5.9,7.1,6.5,6.8,6.7,6.5,6.8,6.1
3,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1993,6.3,6.4,6.2,6.2,6.4,6.9,6.0,5.6,5.6,5.6,5.2,4.8
4,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1994,5.0,5.4,5.3,4.5,4.1,5.6,5.4,5.4,5.1,5.1,5.0,4.2
5,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1995,5.8,6.2,5.6,6.1,6.1,6.3,6.8,5.5,5.5,4.9,4.6,4.4
6,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1996,4.5,4.7,4.1,3.9,3.7,3.9,3.5,3.8,3.8,3.9,3.5,3.5
7,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1997,4.1,4.4,4.1,3.8,3.8,4.7,4.1,4.4,4.0,3.5,2.9,2.8
8,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1998,3.2,3.1,2.8,2.3,2.5,3.3,3.2,3.4,3.7,3.6,3.5,3.8
9,LAUCN010010000000003,"Autauga County, AL",Counties and equivalents,Alabama,unemployment rate,1999,4.0,3.7,3.5,3.5,3.2,4.1,4.2,4.5,4.2,3.8,3.5,3.4
